# Drawing flight routes with NetworkX

### Import

In [ ]:
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import shapely
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

In [ ]:
# Bug fix for cartopy:
# https://github.com/conda-forge/cartopy-feedstock/issues/36
# https://stackoverflow.com/questions/43649159/python-3-4-crashes-when-producing-some-but-not-all-cartopy-maps-with-segment
# When using cartopy downgrade shapely to 1.5.17
# !pip list | grep -i shap
# !pip uninstall shapely; 
# !pip install --no-binary :all: shapely
# !pip install shapely==1.5.17 --force-reinstall
!pip list | grep -i shap

### Data

#### Flights

In [ ]:
names = ('airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment').split(',')
routes = pd.read_csv(
    'https://github.com/ipython-books/'
    'cookbook-2nd-data/blob/master/'
    'routes.dat?raw=true',
    names=names,
    header=None)
routes.sample(3)

#### Airports

The DataFrame index is the IATA code, a 3-characters code identifying the airports.

In [ ]:
names = ('id,name,city,country,iata,icao,lat,lon,'
         'alt,timezone,dst,tz,type,source').split(',')
airports = pd.read_csv(
    'https://github.com/ipython-books/'
    'cookbook-2nd-data/blob/master/'
    'airports.dat?raw=true',
    header=None,
    names=names,
    index_col=4,
    na_values='\\N')
airports_us = airports[airports['country'] ==
                       'United States']
airports_us.sample()

#### Cities with over 300k citizens

<div class='source'> source: https://data.london.gov.uk/dataset/global-city-population-estimates</div>

In [ ]:
cities = pd.read_excel('../_data/global-city-population-estimates.xlsx', 'CITIES-OVER-300K')
cities.sample(5)
cities_dict = cities[['Urban Agglomeration', '2015']].set_index('Urban Agglomeration').to_dict()['2015']
cities_dict;

In [ ]:
cities = pd.read_csv('../_data/airports.csv')
cities.sample(5)
# cities_dict = cities[['Urban Agglomeration', '2015']].set_index('Urban Agglomeration').to_dict()['2015']
# cities_dict

In [ ]:


# http://ourairports.com/data/

#### Missing values

In [ ]:
airports.isnull().sum()
airports.isnull().any().sum()

In [ ]:
airports = airports.dropna()

In [ ]:
airports.sample()

#### DataFrame to dictionary - key is index

- Lookup for geo location  
- Drops NaN's from index!

In [ ]:
airport_geo = airports[['lon', 'lat']].to_dict('index') #.items()
airport_geo['GKA']['lon'], airport_geo['GKA']['lat']

### Create graph

#### Filter data

Let's keep all national US flight routes (source and the destination airports belong to the list of US airports)

In [ ]:
routes_us = routes[
    routes['source'].isin(airports_us.index) & routes['dest'].isin(airports_us.index)]
routes_us.sample()

### Frequency table

In [ ]:
def pivot(y, y_pred, labels=['Negative', 'Positive']):
    dict_labels = {k:v for k, v in zip(np.unique(y), labels)}
    try:
        y_name, y_pred_name = y.name, y_pred.name
    except:
        y_name, y_pred_name = 'True label', 'Predicted label'
    df = pd.DataFrame({y_name: y, y_pred_name: y_pred})  
    df.replace(to_replace=dict_labels, inplace=True)
    return df.groupby([y_name, y_pred_name]).size().unstack(y_pred_name).fillna(0)

In [ ]:
flight_freq = pivot(routes_us.source, routes_us.dest)
flight_freq.head()
flight_freq['DEC'].sum()

#### Frequency counter

In [ ]:
from collections import Counter
flights_from = Counter(routes_us.source)
flights_to = Counter(routes_us.dest)
flights_from_to = flights_from + flights_to
flights_from_to.most_common(5)

#### Sanity check NaN

In [ ]:
routes_us.isnull().sum()

#### Construct the list of edges representing our graph

 - nodes are airports
 - two airports are connected if there exists a route between them (flight network)

In [ ]:
edges = routes_us[['source', 'dest']].values
edges

#### Create the networkX graph from the edges array

In [ ]:
g = nx.from_edgelist(edges)
g.edges();

#### Graph's statistics

- There are 546 US airports and 2781 routes in the dataset.

In [ ]:
print(nx.info(g))

### Visualise graph

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 10))
nx.draw_networkx(g, ax=ax, node_size=5, font_size=6, alpha=.5, width=.5)
ax.set_axis_off()

### Improve visualisation

#### Connected subgraphs

There are a few airports that are not connected to the rest of the airports. 

We keep the largest connected component of the graph as follows:
 - the subgraphs returned by connected_component_subgraphs() are sorted by decreasing size

In [ ]:
sg = next(nx.connected_component_subgraphs(g))

#### Visualise largest connected component subgraph

The graph encodes only the topology (connections between the airports) and not the geometry (actual positions of the airports on a map). Airports at the center of the graph are the largest US airports.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
nx.draw_networkx(sg, ax=ax, with_labels=False, node_size=5, width=.5)
ax.set_axis_off()

#### Draw options

Graphs can be drawn in different ways, see what layouts are available in networkX

In [ ]:
layouts = [eval('nx.'+x) for x in nx.__dir__() if x.endswith('_layout')]
layouts

In [ ]:
# Draw the graph using the random layout
for layout in layouts:
    if layout == nx.rescale_layout: continue
    _ = plt.figure(figsize=(8,7))
    pos = layout(sg)
    _ = nx.draw_networkx(sg, pos);
    _ = plt.gca().set_title(str(layout.__name__));

#### Add geo location(lat lon) to nodes

In [ ]:
ap_loc = {iata:(airport_geo[iata]['lon'], airport_geo[iata]['lat']) for iata in airports_us.index if iata in list(sg.nodes())}

In [ ]:
_ = [sg.add_node(airport, location=geo) for airport, geo in ap_loc.items()]

In [ ]:
list(sg.nodes(data=True))[:5]

#### Visualise graph geographically

In [ ]:
# Draw the graph using custom node positions
plt.figure(figsize=(20,12))

pos = nx.get_node_attributes(sg, 'location')
nx.draw_networkx(sg, pos);

In [ ]:
# Draw the graph adding alpha, removing labels, and softening edge color
plt.figure(figsize=(20,12))

nx.draw_networkx(sg, pos, 
                 node_color='b',
                 alpha=0.5, 
                 with_labels=False, 
                 edge_color='.4')
plt.axis('off')
plt.tight_layout();

#### Add flight frequency between cities to nodes

In [ ]:
ap_loc = {iata:(airport_geo[iata]['lon'], airport_geo[iata]['lat']) for iata in airports_us.index if iata in list(sg.nodes())}

In [ ]:
_ = [sg.add_node(airport, frequency=freq) for airport, freq in flights_from_to.items()if airport in list(sg.nodes())]

In [ ]:
list(sg.nodes(data=True))[:5]

In [ ]:
from collections import Counter
Counter(sorted([(u,v) for (u,v,d) in sg.edges(data=True)])).most_common(10)

In [ ]:
sg.number_of_edges('ABE', 'SFB')

#### Extracting attributes

Using `nx.get_node_attributes` it's easy to extract the node attributes in the graph into DataFrame columns.

In [ ]:
pd.Series(nx.get_node_attributes(sg, 'frequency')).head()

#### Set node size

The node sizes will depend on the degree of the nodes(# of airports connected to every node)

In [ ]:
node_size = [nx.get_node_attributes(sg, 'frequency')[v] for v in sg]

#### Set node color

The node color depends on the node degree.

In [ ]:
deg = nx.degree(sg) # == {v: sg.degree(v) for v in sg}
node_color = [deg[iata] for iata in sg.nodes]
# or
node_color = [sg.degree(n) for n in sg]
node_color = [10*nc/max(node_color) for nc in node_color]
node_color;

The node color depends on theclustering coefficient.

In [ ]:
node_color = [10*(1-c) for c in nx.clustering(sg).values()]
node_color;

#### Set labels

We will display the labels of the largest airports only (at least 20 connections to other US airports)

In [ ]:
labels = {iata: iata if deg[iata] >= 70 else '' for iata in sg.nodes}

#### Select nodes

In [ ]:
min_freq = 200
nodes = [n[0] for n in sg.nodes(data=True) if n[1]['frequency'] > min_freq]

#### Set edge width

In [ ]:
edge_width = [0.002*nx.get_node_attributes(sg, 'frequency')[n] for n in sg]

#### Draw graph

In [ ]:
# Draw graph with varying node color, node size, and edge width
plt.figure(figsize=(20,12))

nx.draw_networkx_nodes(sg, pos, 
                       nodelist=nodes,
                       node_size=node_size, 
                       node_color='b',
                       alpha=0.1,
                       with_labels=False, 
                       width=edge_width, 
                       edge_color='.1',
                       cmap=plt.cm.Blues)

nx.draw_networkx(sg, pos,
                 node_size=node_size, 
                 node_color=node_color, 
                 alpha=0.6, 
                 with_labels=False, 
                 width=edge_width, 
                 edge_color='.4',
                 cmap=plt.cm.Blues)

nx.draw_networkx_labels(sg, pos, labels=labels,font_size=16,font_color='b')

plt.axis('off')
plt.tight_layout();

### Finally Draw the graph on a geographical map

Create a dictionary:
 - keys: airports IATA code
 - value: geometric coordinates

In [ ]:
# pos = {airport: (v['lon'], v['lat']) for airport, v in airports_us.to_dict('index').items()}

#### Use cartopy to project the points on the map:

In [ ]:
# Map projection
crs = ccrs.PlateCarree()  #Mercator() #Geodetic() #Mollweide() #PlateCarree()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20), subplot_kw=dict(projection=crs))
ax.coastlines()

# Extent of continental US. (l,r,b,t)
ax.set_extent([-125, -65, 20, 50])
ax.axis('off')

nx.draw_networkx(sg, ax=ax,
                 font_size=8,
                 alpha=.5,
                 width=edge_width,
                 node_size=20*node_size,
                 pos=pos,
                 node_color=node_color,
                 cmap=plt.cm.Blues);
label_pos = {k: (v[0], v[1]+1) for k, v in pos.items()}

nx.draw_networkx_labels(sg, label_pos, labels=labels, font_size=16, font_color='k');